In [1]:
from qmlab.preprocessing import parse_biomed_data_to_ndarray, reduce_feature_dim
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC

random_state = 12345

X, y, feature_names = parse_biomed_data_to_ndarray("sobar_new")
X = reduce_feature_dim(X, output_dimension=3)
skf = StratifiedKFold(n_splits=5, random_state=random_state, shuffle=True)
clf = SVC(kernel="linear", random_state=random_state)

for train_idx, test_idx in skf.split(X, y):
    X_train = X[train_idx, :]
    X_test = X[test_idx,: ]
    y_train = y[train_idx]
    y_test = y[test_idx]

    clf.fit(X_train, y_train)

    acc_score = clf.score(X_test, y_test)

    print("score", acc_score)
    


score 0.8666666666666667
score 0.8
score 1.0
score 0.9285714285714286
score 0.8571428571428571


In [ ]:
from qmlab.kernel import FidelityQuantumKernel
from qiskit.circuit.library import ZZFeatureMap
from sklearn.model_selection import ShuffleSplit
from qmlab.preprocessing import reduce_feature_dim

X, y, _ = parse_biomed_data_to_ndarray("sobar_new")
qfm = ZZFeatureMap(feature_dimension=X.shape[1], reps=2)
quantum_kernel = FidelityQuantumKernel(feature_map=qfm)
qsvm = SVC(kernel="precomputed")
rs = ShuffleSplit(n_splits=10, test_size=0.3, random_state=random_state)

for train_idx, test_idx in rs.split(X):
    X_train = X[train_idx, :]
    y_train = y[train_idx]
    X_test = X[test_idx, :]
    y_test = y[test_idx]

    kernel_train = quantum_kernel.evaluate_kernel(X_train)
    kernel_test = quantum_kernel.evaluate_kernel(X_test, X_train)
    qsvm.fit(kernel_train, y_train)
    score = qsvm.score(kernel_test, y_test)

    print("score:", score)

score: 0.6363636363636364
score: 0.5909090909090909
score: 0.6363636363636364
score: 0.5909090909090909
score: 0.7272727272727273
score: 0.7272727272727273
score: 0.6818181818181818
score: 0.6363636363636364
score: 0.6363636363636364
score: 0.7727272727272727
